In [90]:
import pandas as pd
import os

In [91]:

def ler_arquivo(caminho_arquivo, tipo):
    try:
        # Lê o arquivo XLSX
        if tipo in ['xls', 'xlsx']:
            df = pd.read_excel(caminho_arquivo)
            return df
        elif tipo == 'csv':
            # Determina o separador dinamicamente
            with open(caminho_arquivo, 'r') as file:
                # Lê as primeiras linhas para determinar o separador
                first_line = file.readline().strip()
                delimiter = ',' if ',' in first_line else ';'
            df = pd.read_csv(caminho_arquivo, sep=delimiter)
            return df
        else:
            raise ValueError("Tipo de arquivo não suportado. Por favor, forneça 'xls', 'xlsx' ou 'csv'.")
    except FileNotFoundError:
        print(f"Arquivo não encontrado: {caminho_arquivo}")
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")

In [92]:
def path_origin_file(file):
    return os.path.dirname(file)

## Mude o nome do arquivo e o indice

In [93]:

nome_do_arquivo = 'tabela_ad_df.csv'
caminho_do_arquivo = ""
folder = 'DF'
if os.path.isdir('CE'):
    caminho_do_arquivo = os.path.join(os.path.abspath(folder),nome_do_arquivo)
else:
    raise Exception("Não Existe o caminho do arquivo")

In [94]:
tipo = caminho_do_arquivo.split('.')[1]
print(tipo)
df = ler_arquivo(caminho_do_arquivo, tipo)
indice_join = "Legenda 1"
df.head()

csv


,Legenda 1,1 Obs:,1 Nivel1,1 Nivel2,1 Nivel3,1 Nivel4,1 Textura1,1 Textura2,1 Textura3,1 Pedregosidade,...,5 Dtext3,5 ADtex1,5 ADtex2,5 ADtex3,5 Adsolo,5 Psolo,5 ADprop,ADUM,Unnamed: 152,Unnamed: 153
0,Ade,NaN,NEOSSOLOS,FLÚVICOS,Distrófico e Eutrófico,NaN,Argilosa,NaN,NaN,,...,0,NaN,NaN,NaN,"0,000",NaN,"0,00000","1,44200",NaN,NaN
1,AQd1,NaN,NEOSSOLOS,QUARTZARÊNICO,NaN,NaN,Arenosa,NaN,NaN,,...,0,NaN,NaN,NaN,"0,000",NaN,"0,00000","0,57000",NaN,NaN
2,AQd2,NaN,NEOSSOLOS,QUARTZARÊNICO,NaN,NaN,Arenosa,NaN,NaN,,...,0,NaN,NaN,NaN,"0,000",NaN,"0,00000","0,57000",NaN,NaN
3,BV,NaN,CHERNOSSOLO,AVERMELHADO,,NaN,Argilosa,NaN,NaN,,...,0,NaN,NaN,NaN,"0,000",NaN,"0,00000","0,88200",NaN,NaN
4,Cd1,NaN,CAMBISSOLO,HISTICO/HÁPLICO/HUMICO,Tb,NaN,Argilosa,NaN,NaN,,...,0,NaN,NaN,NaN,"0,000",NaN,"0,00000","1,34000",NaN,NaN


## Rename Colunas

In [95]:
colunas_drop = []
textura_componentes = {'textura_c1': [], 'textura_c2': [], 'textura_c3': [], 'textura_c4': [],'textura_c5': [],}
solos_componentes = {'solos_c1': [], 'solos_c2': [], "solos_c3":[],"solos_c4":[],"solos_c5":[]}

for coluna in df.columns:
    lowerString = coluna.lower()
    if "pedre" in lowerString:
        df.rename(columns={coluna:f"pedrego_c{coluna[0]}"}, inplace=True)
    elif "nivel" in lowerString:
        df[coluna] = df[coluna].astype(str)
        if(int(coluna[0]) and int(coluna[-1])):
            df.rename(columns={coluna: f'{coluna[0]} Nivel{coluna[-1]}'}, inplace=True)
            solos_componentes[f'solos_c{coluna[0]}'].append(f'{coluna[0]} Nivel{coluna[-1]}')
    elif "rocho" in lowerString:
        df.rename(columns={coluna:f"rocho_c{coluna[0]}"}, inplace=True)
    elif "relevo" in lowerString:
        df.rename(columns={coluna:f"relevo_c{coluna[0]}"}, inplace=True)
    elif "adum" in lowerString:
        df.rename(columns={coluna:f"ad_um"}, inplace=True)
    elif "solo_" in lowerString:
        continue
    elif "ad_" in lowerString:
        continue
    elif indice_join.lower() == lowerString:
        df.rename(columns={coluna:'ind_csv'}, inplace=True)
        
    elif "ad solo" in lowerString or "adsolo" in lowerString:
        df.rename(columns={coluna:f"ad_c{coluna[0]}"}, inplace=True)
        continue
    elif "textu" in lowerString:
        df[coluna] = df[coluna].astype(str)
        if(coluna.startswith("1") and not coluna in textura_componentes["textura_c1"]):
            textura_componentes["textura_c1"].append(coluna)
        if(coluna.startswith("2") and not coluna in textura_componentes["textura_c2"]):
            textura_componentes["textura_c2"].append(coluna)
        if(coluna.startswith("3")) and not coluna in textura_componentes["textura_c3"]:
            textura_componentes["textura_c3"].append(coluna)
        if(coluna.startswith("4")) and not coluna in textura_componentes["textura_c4"]:
            textura_componentes["textura_c4"].append(coluna)
        if(coluna.startswith("5")) and not coluna in textura_componentes["textura_c5"]:
            textura_componentes["textura_c5"].append(coluna)
    else:
        colunas_drop.append(coluna)

## Solos


In [96]:

for key, value in solos_componentes.items():
    df[key] = df[value].apply(lambda row: ' '.join(filter(lambda x: x if x !="nan" else None, row)), axis=1)
    df.drop(columns=value, inplace=True)

## Textura

In [97]:
for key, value in textura_componentes.items():
    df[key] = df[value].apply(lambda row: ', '.join(filter(lambda x: x if x !="nan" else None, row)), axis=1)
    df.drop(columns=value, inplace=True)

## c1_class

In [98]:
df['c1_class'] = df['solos_c1']+' - ' + df['ind_csv']

## Limpando dataframe

In [99]:
colunas_drop
df.drop(columns=colunas_drop, inplace=True)


## Exportando

In [100]:
nome_do_arquivo_out = nome_do_arquivo.split('.')[0]+'_novo'+'.csv'

path_saida = os.path.join(path_origin_file(caminho_do_arquivo),nome_do_arquivo_out)
df.to_csv(path_saida, index=False, encoding='utf-8-sig', sep=',')